In [1]:
import math
import time
import pandas as pd
import numpy as np
import numpy.linalg as la

from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.multioutput import MultiOutputRegressor

In [2]:
def load_data(path):
    df = pd.read_csv(path)
    data = df.drop(columns=['Unnamed: 0'])
    
    return data

In [3]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate)
    train_data = data[0:train_size]
    test_data = data[train_size:time_len]
    
    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len):
        a = train_data[i: i + seq_len + pre_len]
        trainX.append(a[0 : seq_len])
        trainY.append(a[seq_len : seq_len + pre_len])
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
        
    return trainX, trainY, testX, testY

In [4]:
def getTestY(data):
    tmp_scaler = StandardScaler()
    tmp_data = tmp_scaler.fit_transform(data)
    tmp_data = tmp_scaler.inverse_transform(tmp_data)
    _, _, _, testY = preprocess_data(tmp_data, data.shape[0], train_rate, seq_len, pre_len)
    testY = np.array(testY)
    testY = np.reshape(testY, [-1, pre_len])
    
    return testY

In [5]:
 def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    mape = mean_absolute_percentage_error(a, b)
    F_norm = la.norm(a-b)/la.norm(a)
    
    return rmse, mae, mape, 1-F_norm

In [6]:
def predictSVR(data, testY):
    scaler = StandardScaler()
    data = scaler.fit_transform(data)
    
    rmses, maes, mapes, accs = [], [], [], []
    for i in range(data.shape[1]):
        print('Node', i)
        start = time.time()
        a = data[:, i]
        aX, aY, tX, tY = preprocess_data(a, data.shape[0], train_rate, seq_len, pre_len)
        
        aX = np.array(aX)
        aX = np.reshape(aX, [-1, seq_len])
        aY = np.array(aY)
        aY = np.reshape(aY, [-1, pre_len])

        tX = np.array(tX)
        tX = np.reshape(tX, [-1, seq_len])
        tY = np.array(tY)
        tY = np.reshape(tY, [-1, pre_len])
        
        print('(', aX.shape, aY.shape, tX.shape, tY.shape, round(aX.mean(),3), ')')
        reg = MultiOutputRegressor(SVR(kernel='linear'))
        reg.fit(aX, aY)
        pred = reg.predict(tX)
        
        mean = scaler.mean_[i]
        std = np.sqrt(scaler.var_[i])
        pred = pred*std + mean
        tY = tY*std + mean
        if i==data.shape[1]-1: tY = testY[tY.shape[0]*i:]
        else: tY = testY[tY.shape[0] * i:tY.shape[0] * (i+1)]
        
        rmse, mae, mape, acc = evaluation(tY, pred)
        rmses.append(rmse)
        maes.append(mae)
        mapes.append(mape)
        accs.append(acc)
        
        print('(', rmse, mae, mape, acc, time.time() - start)
    
    print('RMSE: ' + str(np.mean(rmses)) + ', MAE: ' + str(np.mean(maes)) + ', MAPE: ' + str(np.mean(mapes)) + ', ACC: ' + str(np.mean(accs)))

In [7]:
train_rate = 0.8
seq_len = 12
pre_len = 12

In [8]:
path_0 = '../Data/Gangnam/speed_gangnam_0.csv'
path_5 = '../Data/Gangnam/speed_gangnam_5.csv'
path_10 = '../Data/Gangnam/speed_gangnam_10.csv'
path_20 = '../Data/Gangnam/speed_gangnam_20.csv'

In [9]:
masterData = load_data(path_0)
print(masterData.shape, (masterData == 0).sum().sum())
testY = getTestY(masterData)
testY.shape, (testY == 0).sum().sum()

(2880, 506) 0


((279312, 12), 0)

### Unobserved Nodes = 0%

In [10]:
data = load_data(path_0)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 0
Node 0
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.004 )
( 11.09762919988002 7.825636977922009 0.25266995489672706 0.6572049390540399 5.19910192489624
Node 1
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.033 )
( 13.608090470027502 10.101243139987213 0.32168827645829207 0.5814480386312941 5.005621433258057
Node 2
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.098 )
( 12.055628969634993 9.327033468495763 0.33621863122193435 0.6303553299452874 5.456418991088867
Node 3
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 14.833984283979696 12.667538402817101 0.5040888027804368 0.5469465461124852 6.976353883743286
Node 4
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 16.548692604080756 12.895686780867218 0.37811738836569336 0.4955803732864206 6.842714309692383
Node 5
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.018 )
( 13.5909650879623 9.898567022596275 0.2860004167501784 0.5859203771393189 8.119297742843628
Node 6
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.013 )


( 19.180597889922165 16.1393010260404 0.46666357262846264 0.4221842232266396 5.676824569702148
Node 54
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.009 )
( 17.451248989709146 14.195713858050878 0.4084582349370687 0.47269331140386417 6.352025032043457
Node 55
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.008 )
( 10.736110037698607 7.735044537158419 0.2669174087690284 0.6753759804716724 8.128318548202515
Node 56
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.003 )
( 13.26030628518487 10.65717161220869 0.4209322146655059 0.5999313805846873 5.998966932296753
Node 57
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 11.561360977996305 8.506769810867048 0.295413714314424 0.6503130659340111 5.855350494384766
Node 58
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.037 )
( 15.674469769449518 13.294903549769595 0.5326900889133145 0.524421371748812 5.650897026062012
Node 59
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 11.855580567987035 9.263797993531979 0.3558973505806864 0.6399388

( 9.135365621734831 6.762358275291629 0.3202599476817554 0.663673355273326 6.2682459354400635
Node 107
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.027 )
( 13.440489189323738 11.821182949507104 0.6021180954648524 0.5039016868976846 6.163529634475708
Node 108
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.019 )
( 9.864327508627555 7.086421556283401 0.29422053792399805 0.6334070371889533 7.021233320236206
Node 109
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.011 )
( 10.47373138237033 7.867334155210944 0.34540003430066885 0.6095792824267315 6.31013560295105
Node 110
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.032 )
( 10.128107448771225 7.830640301635332 0.3875799329031088 0.6222098341709825 6.358009576797485
Node 111
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 9.683441264145099 7.008802477682323 0.3108661142760238 0.6386992773411043 5.835442781448364
Node 112
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.027 )
( 9.877403714875497 7.445535666974382 0.36272958192542987 0.629

( 10.903828467952556 7.783542418810946 0.3252946298411542 0.5833804873810549 7.662520885467529
Node 160
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.032 )
( 9.952120320465276 7.3378958019456855 0.37108387206479226 0.6198547686063618 7.524889707565308
Node 161
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.041 )
( 10.09244131308749 7.47305436850353 0.3832966688043606 0.6149407120214514 7.466045379638672
Node 162
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.003 )
( 11.986975589128205 8.796041949610133 0.376355184537406 0.542844769431511 5.32477068901062
Node 163
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.027 )
( 10.257291605317466 7.581347601066511 0.3835115081767228 0.6096930373359961 5.698769569396973
Node 164
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.003 )
( 12.295855345693044 9.61349900610624 0.5048034201017746 0.5320372441034549 6.491650104522705
Node 165
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.007 )
( 11.707444918235145 9.291990469547315 0.47986394149558964 0.5540

( 13.303705340433904 10.209773568853903 0.39061143389968356 0.541529763586621 4.433651685714722
Node 213
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 17.14520546067382 15.423769734866392 0.7151064060245651 0.4086975280961205 6.759935140609741
Node 214
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.04 )
( 15.521253777967608 12.331770526328958 0.4084221614401535 0.4650830098005163 5.915219068527222
Node 215
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.085 )
( 12.213762507547791 8.80833223264194 0.29383028980464804 0.5794626836896744 5.547028541564941
Node 216
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.068 )
( 10.178398792928782 7.239226318128512 0.28292782473848943 0.6501706591027946 5.972071170806885
Node 217
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.022 )
( 17.88664380597103 16.18337845325153 0.7524386824931835 0.38691301823429103 6.075729846954346
Node 218
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.057 )
( 11.794202735247278 8.538406128572577 0.3121884287858851 0.59

( 13.892913215025434 10.148112239576477 0.2866339722302383 0.5835803190265227 6.995336055755615
Node 266
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 17.249340125383224 13.724759231568605 0.3878784176213117 0.48428213627107763 6.093699932098389
Node 267
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.002 )
( 14.218645784748864 10.523268085975467 0.31522953597425546 0.5763500471021512 5.575070858001709
Node 268
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.027 )
( 13.371451348459555 9.587494852492235 0.27811306994607254 0.6020323352386288 5.919224500656128
Node 269
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.005 )
( 12.71090237933428 9.28807159433501 0.29424267423216577 0.6225141006087129 7.466998338699341
Node 270
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.036 )
( 20.60031778460826 17.178840219774752 0.4944881160872187 0.38986436463308727 5.461998224258423
Node 271
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.001 )
( 13.047607888520716 9.579617974183188 0.3073515929558

( 19.180428710471965 15.693369313117273 0.4589923645900132 0.42600782847386043 6.643447399139404
Node 319
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.059 )
( 17.006963401960057 13.448659058943896 0.38983952278309303 0.4912109859305026 5.315794467926025
Node 320
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.097 )
( 15.420874260950983 11.821809957240186 0.34674844191818455 0.5379120474542463 5.280888080596924
Node 321
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.051 )
( 12.524586373415753 9.195544890642514 0.2966861686936391 0.6230532639009487 6.288208723068237
Node 322
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.007 )
( 13.433516322382815 10.192283720755727 0.34587791302127235 0.5953589404804935 4.69149112701416
Node 323
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.01 )
( 15.230176111891625 11.641986398354454 0.3547171973229562 0.5407620747914414 5.074161529541016
Node 324
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.021 )
( 12.43416528272626 9.547991435206521 0.3484186612070164 

( 12.382537064430334 9.900428207655283 0.4773710866444949 0.5423671990669316 5.672858476638794
Node 372
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 19.271710044994272 17.578285560458124 0.8991599872819397 0.2875145991365844 6.224314451217651
Node 373
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.041 )
( 10.952793327510426 7.763162355468158 0.30481556964864504 0.5935434518025023 5.271911859512329
Node 374
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.009 )
( 15.213944085759758 13.204183400193521 0.6825654978738179 0.4353702803294357 6.166518449783325
Node 375
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.012 )
( 38.288563149961966 36.25891050449335 1.7973905883892598 -0.42542567045552215 4.758312940597534
Node 376
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 58.38676955096747 57.53647889935902 2.7912506686723924 -1.1816806796755812 5.089449882507324
Node 377
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.033 )
( 42.30393801751553 38.70467343698585 1.927509872192778

( 10.852871297518226 8.639838028372887 0.49955436539526193 0.5750509945942279 6.004987001419067
Node 425
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.006 )
( 12.590799978247459 10.251762153421849 0.6020738018563098 0.5081181383824398 5.384606838226318
Node 426
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.023 )
( 22.951573364805125 21.020457082117602 1.210308003469824 0.10827763315078898 5.0016326904296875
Node 427
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.074 )
( 27.830665936271988 25.186427182849027 1.4120244361520822 -0.08039957431620137 4.143922805786133
Node 428
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.015 )
( 32.845329174922014 30.743046681643932 1.7456610265502286 -0.2849673596489015 4.543860197067261
Node 429
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.012 )
( 34.89953503735508 32.23491121616991 1.8015701087473548 -0.3638470999493608 3.675170660018921
Node 430
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.03 )
( 38.11724142286757 36.6736671115975 2.065522720837741

( 10.33799051801794 7.280838397976279 0.2628655025119508 0.6494777624584069 5.536202907562256
Node 478
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.002 )
( 12.248892879489896 9.008980264923645 0.30721986202454116 0.5845220043266531 5.342721223831177
Node 479
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.019 )
( 10.315160282831316 7.624304290170074 0.30103023634103343 0.6498427113176977 5.341723442077637
Node 480
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.063 )
( 14.674554243313104 11.236194031846479 0.38032460355418163 0.5035367573792495 4.480031728744507
Node 481
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.101 )
( 10.040748201722351 6.969095208626698 0.2493540801166593 0.6603034965289334 5.9919819831848145
Node 482
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.085 )
( 16.258829167982707 13.095017293568143 0.4256771841038826 0.4499277233428294 5.865293264389038
Node 483
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.05 )
( 10.810191249413936 7.703414024551633 0.27182132420471583 

### Unobserved Nodes = 5%

In [11]:
data = load_data(path_5)
print(data.shape, (data == 0).sum().sum())
predictSVR(data, testY)

(2880, 506) 72000
Node 0
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.004 )
( 11.09762919988002 7.825636977922009 0.25266995489672706 0.6572049390540399 5.202096700668335
Node 1
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.033 )
( 13.608090470027502 10.101243139987213 0.32168827645829207 0.5814480386312941 4.4012370109558105
Node 2
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.098 )
( 12.055628969634993 9.327033468495763 0.33621863122193435 0.6303553299452874 4.640625953674316
Node 3
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 14.833984283979696 12.667538402817101 0.5040888027804368 0.5469465461124852 5.093417167663574
Node 4
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 16.548692604080756 12.895686780867218 0.37811738836569336 0.4955803732864206 4.974674224853516
Node 5
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.018 )
( 13.5909650879623 9.898567022596275 0.2860004167501784 0.5859203771393189 6.074812889099121
Node 6
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.

( 19.180597889922165 16.1393010260404 0.46666357262846264 0.4221842232266396 4.676493406295776
Node 54
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.009 )
( 17.451248989709146 14.195713858050878 0.4084582349370687 0.47269331140386417 5.138278007507324
Node 55
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.008 )
( 10.736110037698607 7.735044537158419 0.2669174087690284 0.6753759804716724 6.485634803771973
Node 56
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.003 )
( 13.26030628518487 10.65717161220869 0.4209322146655059 0.5999313805846873 4.9418206214904785
Node 57
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.007 )
( 11.561360977996305 8.506769810867048 0.295413714314424 0.6503130659340111 4.878932952880859
Node 58
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.037 )
( 15.674469769449518 13.294903549769595 0.5326900889133145 0.524421371748812 4.648608684539795
Node 59
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 11.855580567987035 9.263797993531979 0.3558973505806864 0.639938

( 13.440489189323738 11.821182949507104 0.6021180954648524 0.5039016868976846 5.389595985412598
Node 108
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 26.90812020227441 25.41303140096619 1.0 0.0 0.011968135833740234
Node 109
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.011 )
( 10.47373138237033 7.867334155210944 0.34540003430066885 0.6095792824267315 5.397602796554565
Node 110
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.032 )
( 10.128107448771225 7.830640301635332 0.3875799329031088 0.6222098341709825 5.6977221965789795
Node 111
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.018 )
( 9.683441264145099 7.008802477682323 0.3108661142760238 0.6386992773411043 5.158245086669922
Node 112
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.027 )
( 9.877403714875497 7.445535666974382 0.36272958192542987 0.6299924047941312 5.341694355010986
Node 113
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 11.083685673726256 8.45277607018461 0.38115269583299166 0.5842516192649901 4.776262521743

( 10.09244131308749 7.47305436850353 0.3832966688043606 0.6149407120214514 5.181190252304077
Node 162
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.003 )
( 11.986975589128205 8.796041949610133 0.376355184537406 0.542844769431511 4.281550884246826
Node 163
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.027 )
( 10.257291605317466 7.581347601066511 0.3835115081767228 0.6096930373359961 4.6236114501953125
Node 164
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.003 )
( 12.295855345693044 9.61349900610624 0.5048034201017746 0.5320372441034549 5.320808172225952
Node 165
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.007 )
( 11.707444918235145 9.291990469547315 0.47986394149558964 0.5540952724779387 4.409225940704346
Node 166
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.064 )
( 17.465537930915527 14.47979920122853 0.7511744029474364 0.3364880027719882 3.5415256023406982
Node 167
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.061 )
( 17.816209794715263 14.466867991687621 0.8035931143863815 0.318

( 15.521253777967608 12.331770526328958 0.4084221614401535 0.4650830098005163 5.273937225341797
Node 215
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.085 )
( 12.213762507547791 8.80833223264194 0.29383028980464804 0.5794626836896744 4.514902353286743
Node 216
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.068 )
( 10.178398792928782 7.239226318128512 0.28292782473848943 0.6501706591027946 4.810144901275635
Node 217
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.022 )
( 17.88664380597103 16.18337845325153 0.7524386824931835 0.38691301823429103 4.963733434677124
Node 218
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.057 )
( 11.794202735247278 8.538406128572577 0.3121884287858851 0.5957802024582175 4.43614387512207
Node 219
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.03 )
( 12.270534748048272 9.933281138377554 0.4469515683228395 0.5791416412220718 4.5867414474487305
Node 220
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.006 )
( 12.750633556252817 10.202945378943989 0.4557573444453542 0.5

( 14.218645784748864 10.523268085975467 0.31522953597425546 0.5763500471021512 4.52490758895874
Node 268
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.027 )
( 13.371451348459555 9.587494852492235 0.27811306994607254 0.6020323352386288 5.042522668838501
Node 269
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.005 )
( 12.71090237933428 9.28807159433501 0.29424267423216577 0.6225141006087129 6.1346049308776855
Node 270
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.036 )
( 20.60031778460826 17.178840219774752 0.4944881160872187 0.38986436463308727 4.414200067520142
Node 271
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.001 )
( 13.047607888520716 9.579617974183188 0.30735159295584225 0.6145817390191877 4.823109865188599
Node 272
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.004 )
( 11.809090663926732 8.547738700852937 0.2780883544021747 0.6516095974597004 5.585074186325073
Node 273
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 33.999498031292035 32.12818689613526 1.0 0.0 0.010969877

( 15.420874260950983 11.821809957240186 0.34674844191818455 0.5379120474542463 4.495983362197876
Node 321
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.051 )
( 12.524586373415753 9.195544890642514 0.2966861686936391 0.6230532639009487 5.260939359664917
Node 322
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.007 )
( 13.433516322382815 10.192283720755727 0.34587791302127235 0.5953589404804935 4.054164886474609
Node 323
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.01 )
( 15.230176111891625 11.641986398354454 0.3547171973229562 0.5407620747914414 4.386309623718262
Node 324
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.021 )
( 12.43416528272626 9.547991435206521 0.3484186612070164 0.6240186971739046 4.679492712020874
Node 325
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.034 )
( 11.956106102556015 9.39462537707718 0.37743015316391676 0.6383849435652063 4.8779616355896
Node 326
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.028 )
( 11.155271299364989 7.9855423747771255 0.2798936297703925 0.66

( 10.952793327510426 7.763162355468158 0.30481556964864504 0.5935434518025023 4.33042573928833
Node 374
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.009 )
( 15.213944085759758 13.204183400193521 0.6825654978738179 0.4353702803294357 4.995652675628662
Node 375
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.012 )
( 38.288563149961966 36.25891050449335 1.7973905883892598 -0.42542567045552215 3.847712993621826
Node 376
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.001 )
( 58.38676955096747 57.53647889935902 2.7912506686723924 -1.1816806796755812 4.048181056976318
Node 377
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.033 )
( 42.30393801751553 38.70467343698585 1.9275098721927781 -0.5826293125372408 2.7426698207855225
Node 378
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.001 )
( 33.65808885624941 30.539786229759766 1.5527101316550915 -0.26139703314339324 3.075777530670166
Node 379
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.005 )
( 10.901669415413496 8.281573103433052 0.3988239223009

( 22.951573364805125 21.020457082117602 1.210308003469824 0.10827763315078898 3.9813590049743652
Node 427
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.074 )
( 27.830665936271988 25.186427182849027 1.4120244361520822 -0.08039957431620137 3.2842233180999756
Node 428
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.015 )
( 32.845329174922014 30.743046681643932 1.7456610265502286 -0.2849673596489015 3.699113607406616
Node 429
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.012 )
( 34.89953503735508 32.23491121616991 1.8015701087473548 -0.3638470999493608 3.0459039211273193
Node 430
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.03 )
( 38.11724142286757 36.6736671115975 2.065522720837741 -0.48771342696233355 4.743278503417969
Node 431
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.004 )
( 27.88068432380577 26.14937495408897 1.5085946094167457 -0.08877858989850118 5.392587661743164
Node 432
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.027 )
( 12.133994417114806 8.71435551419631 0.350237661485359

( 14.674554243313104 11.236194031846479 0.38032460355418163 0.5035367573792495 4.195782423019409
Node 481
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.101 )
( 10.040748201722351 6.969095208626698 0.2493540801166593 0.6603034965289334 5.4324517250061035
Node 482
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.085 )
( 16.258829167982707 13.095017293568143 0.4256771841038826 0.4499277233428294 5.099368333816528
Node 483
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.05 )
( 10.810191249413936 7.703414024551633 0.27182132420471583 0.6339679814025876 4.624640464782715
Node 484
( (2280, 12) (2280, 12) (552, 12) (552, 12) 0.0 )
( 29.543041194694247 27.96842391304347 1.0 0.0 0.010970592498779297
Node 485
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.014 )
( 9.7963679298788 7.022013881097966 0.27393524493351395 0.6690859339943768 5.647938013076782
Node 486
( (2280, 12) (2280, 12) (552, 12) (552, 12) -0.027 )
( 10.848542420377326 7.629255700510222 0.2815078191327928 0.6326829868563322 4.5119426250